In [93]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [94]:
netflix_rating = pd.read_csv("netflix_ratings_data.csv")

In [95]:
all_genres = netflix_rating['genres'].str.split("\\s*,\\s*").explode(ignore_index=True).drop_duplicates()
all_genres = all_genres.sort_values()
genres = all_genres.to_list()
all_genres.to_csv('netflix_genres_data.csv', sep=',', index=False)

In [96]:
netflix_genre = pd.DataFrame([netflix_rating.title, netflix_rating.genres]).transpose()
netflix_genre['genres'] = netflix_genre['genres'].str.split("\\s*,\\s*")
netflix_genre['score'] = netflix_rating['averageRating']
netflix_genre

,title,genres,score
0,dick johnson is dead,"[Biography, Documentary, Drama]",7.4
1,ganglands,"[Action, Crime, Drama]",7.0
2,jailbirds new orleans,"[Documentary, Reality-TV]",6.6
3,midnight mass,"[Drama, Fantasy, Horror]",7.7
4,my little pony: a new generation,"[Adventure, Animation, Comedy]",6.9
...,...,...,...
6415,zindagi gulzar hai,[Romance],9.0
6416,zodiac,"[Crime, Drama, Mystery]",7.7
6417,zombieland,"[Action, Comedy, Horror]",7.6
6418,zoom,"[Action, Adventure, Comedy]",4.3


In [101]:
#Reference: https://www.analyticsvidhya.com/blog/2019/04/predicting-movie-genres-nlp-multi-label-classification/?fbclid=IwAR3L_2ZH4mMH3TDtU3Xo70o5I5oVvPu25god32CR3IilRz_1MZOzOvzkmbs
mlb = MultiLabelBinarizer()
genre_mlb = mlb.fit(netflix_genre['genres'])

# transform target variable
y = mlb.transform(netflix_genre['genres'])

# movie_genres_mlb = pd.DataFrame(y, columns = genres)
# movie_genres_mlb['title'] = netflix_genre['title']
# movie_genres_mlb['score'] = netflix_rating['averageRating']
# movie_genres_mlb = movie_genres_mlb.set_index('title')
y

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [106]:
X_train, X_valid, y_train, y_valid = train_test_split(y, netflix_genre['score'], test_size=0.2, random_state=9)
model = RandomForestClassifier(n_estimators=100, max_depth=3, min_samples_leaf=10)
model.fit(X_train, y_train)
# print(model.score(X_train, y_train))
# print(model.score(X_valid, y_valid))

ValueError: Unknown label type: 'continuous'